In [25]:
# CSV Data Ingestion

import pandas as pd
import glob
import shutil
import os
from dotenv import load_dotenv
load_dotenv(override=True)
import pyodbc 
import numpy as np

Concat all CSVs in data file to one df

In [26]:
path = 'Data'
csv_files = glob.glob(path+"/*.csv")

df_raw_data = pd.DataFrame()

for csv in csv_files:
    frame = pd.read_csv(csv)
    frame['SourceFile'] = os.path.basename(csv)
    df_raw_data = pd.concat([df_raw_data,frame])
    


Move files to archive

In [27]:
source_dir = 'Data'
target_dir = 'Data/Old'
    
file_names = os.listdir(source_dir)
    
for file_name in file_names:
    shutil.move(os.path.join(source_dir, file_name), target_dir)

Clean column names for whitespace

In [28]:
df_raw_data.columns = [c.replace(' ', '') for c in df_raw_data.columns]

Replace Nulls

In [29]:
df_raw_data_clean = df_raw_data.replace(np.nan, '', regex=True)

Move files to DB

In [30]:

server = os.getenv('SQL_SERVER')
database = os.getenv('SQL_DATABASE')
cnxn_str = 'DRIVER={SQL Server};server='+server+';Database='+database+';Trusted_Connection=yes;'
cnxn = pyodbc.connect(cnxn_str)
cursor = cnxn.cursor()


In [31]:
for index, row in df_raw_data_clean.iterrows():
    cursor.execute("Insert into raw_txns (BSBNumber,AccountNumber,TransactionDate,Narration,Cheque,Debit,Credit,Balance,TransactionType,SourceFile) Values (?,?,?,?,?,?,?,?,?,?)", row.BSBNumber,row.AccountNumber,row.TransactionDate,row.Narration,row.Cheque,row.Debit,row.Credit,row.Balance,row.TransactionType,row.SourceFile)
cnxn.commit()

In [ ]:
cursor.execute("EXEC etl.RawToStd")
cnxn.commit()